<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/SalesMailPlace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mecab-python3
!pip install mojimoji

     |████████████████████████████████| 15.9MB 3.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [0]:
import re
import MeCab
import mojimoji

def serialize(words):
    #大文字統一
    #words = words.upper()

    #英数字のみ半角統一
    words = mojimoji.zen_to_han(words, kana=False)
      
    #改行コード変換
    words = words.replace("\r\n","")

    return words

In [0]:
def tanka_pickup(str_wk):
  #======================================================
  #スキップワード削除
  #======================================================
  del_word = ["<",">","＜","＞","[","]","《","》","≪","≫","【", "】","(",")","（","）"," ","　",":","："]
  
  #路線名パターン
  line_ptn_list = ['JR([ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)+',
            '東武([ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)+',
            '西武([ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)+',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)+線',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)+鉄道',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)+メトロ',
            '([ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)+ライン'
            ]

  #路線名削除
  for line in line_ptn_list:
    del_line_word = re.search(line,str_wk)
    if del_line_word != None:
      str_wk = str_wk.replace(del_line_word.group(),"")
  
  base_word = '場所'
  tanka_yuragi = ["最寄駅","最寄","作業場所","勤務地"]
  
  #不要文字列削除
  for word in del_word:
    str_wk = str_wk.replace(word,'')

  #ゆらぎ統一
  for tanka in tanka_yuragi:
    str_wk = str_wk.replace(tanka,base_word)
  
  #正規化
  str_wk = serialize(str_wk)
    
  #======================================================
  #分かち書き
  #======================================================
  tagger = MeCab.Tagger("-Owakati")
  list_output = (tagger.parse(str_wk)).split(' ')

  #======================================================
  #「単価」に最も近い位置で書かれている数字が含まれる文節を抽出
  #======================================================
  tanka_str = ""
    
  if list_output.count(base_word) > 0:
    if re.search('[ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)',list_output[list_output.index(base_word)+1]) != None:
      tanka_str = re.search('[ぁ-ゟ]+|[\ァ-ヿ]+|[\u4E00-\u9FD0]+)',list_output[list_output.index(base_word)+1])
      tanka_str = tanka_str.group()
    else:
      tanka_str = "勤務地情報あり"
  else:
    tanka_str = "勤務地情報なし"
    
  return tanka_str

In [13]:
import pandas as pd
import codecs

#utf-8で作られていないファイルの場合のCSV読み込み方法
with codecs.open("python.CSV", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")

df2 = df[["本文"]]

#行単位の繰り返し
for index,item in df2.iterrows():
  str_wk = item['本文']
  pickup_str = tanka_pickup(str_wk)
  print(str(index) + ":" + pickup_str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: read_table is deprecated, use read_csv instead.
  


0:勤務地情報なし
1:勤務地情報なし
2:勤務地情報なし
3:勤務地情報なし


error: ignored